# Entscheidungsbaumklassifikation mit Scikit-learn

Ein Entscheidungsbaummodell leitet aus den Merkmalswerten der Beobachtungen Entscheidungsregeln ab, mit deren Hilfe sich die Werte des Zielmerkmals bzw. die Klassenzugehörigkeit der Beobachtungen bestimmen lassen.

## Vorbereitung des Datensatzes

Im Folgenden arbeiten wir mit dem <a href="https://archive.ics.uci.edu/ml/datasets/Wine">_Wine Recognition Dataset_</a>, das Ergebnisse einer chemischen Analyse von 178 Weinen drei verschiedener Rebsorten enthält. Bei der Analyse wurden die Mengen von 13 Inhaltsstoffen wie z.B. Alkohol, Apfelsäure und Magnesium bestimmt, die in den Weinen dieser drei Sorten enthalten sind. Die Merkmalswerte im Datensatz repräsentieren die in den Weinen gemessenen Inhaltsstoffmengen. Zumindest stimmt das für die meisten Merkmale, da für die Merkmale Farbtiefe und Farbton keine ausreichende Beschreibung vorhanden ist. Die Klassifikationsaufgabe bei diesem Datensatz besteht darin, anhand der Inhaltsstoffe eines Weins seine Rebsorte zu bestimmen.

Zuerst werden alle benötigten Module in das Programm importiert.

In [ ]:
# Importiere die nötigen Module
from sklearn import tree, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

Der _Wine_-Datensatz kann mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html">_load\_wine()_</a> geladen werden, die im _Scikit-learn_-Untermodul _Data Sets_ zur Verfügung steht. Damit die Werte der beschreibenden Merkmale und des Zielmerkmals in getrennte _ndarrays_ geladen werden, wird der Parameter _return\_X\_y_ auf _True_ gesetzt.

In [ ]:
# Lade den Wine Datensatz
X_wine, y_wine = datasets.load_wine(return_X_y=True)

Die Beobachtungen bzw. Weine werden durch ihre Werte für die Merkmale Alkoholgehalt, Apfelsäuregehalt und Flavonoidgehalt in einem 3-dimensionalen Diagramm grafisch dargestellt. Die Rebsorte der Weine wird durch unterschiedliche Farben der Datenpunkte markiert.

In [ ]:
pyo.init_notebook_mode()
fig1 = px.scatter_3d(x=X_wine[:,0], y=X_wine[:,1],
                     z=X_wine[:,6], color=y_wine.astype(str))
fig1.show()

Bevor ein Entscheidungsbaummodell erstellt wird, wird der Datensatz in eine Trainings- und eine Testmenge zerlegt. Dafür wird die Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">_train\_test\_split()_</a> des Untermoduls _Model Selection_ von _Scikit-learn_ benutzt. Diese Funktion erwartet als Eingabe die Datenmatrix mit dem entsprechenden Zielmerkmal. Mit dem Parameter _test\_size_ wird die Größe der Testmenge festgelegt. Mit der Wertzuweisung des Parameters _random\_state_ wird die Reproduzierbarkeit der Zerlegung gesichert. In den Voreinstellungen der Funktion ist festgelegt, dass die Beobachtungen vor der Zerlegung gemischt werden. Für diesen Datensatz ist es besonders wichtig, diese Voreinstellung beizubehalten, weil die Beobachtungen in der ursprünglichen Datenmatrix nach dem Zielmerkmal  sortiert sind.

In [ ]:
# Teile den Datensatz in die Trainings- und die Testmenge auf
X_train, X_test, y_train, y_test = train_test_split(
    X_wine, y_wine, test_size=0.1, random_state=45)

## Entscheidungsbaummodell

Ein Entscheidungsbaummodell wird mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html">_DecisionTreeClassifier()_</a> des _Scikit-learn_-Untermoduls <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree">_Decision Trees_</a> erstellt. Das Entscheidungsbaummodel kann durch viele Parameter der Funktion vorbestimmt werden. Im Folgenden werden nur die Wichtigsten von ihnen beschrieben.
* Mit dem Parameter _criterion_ kann die Funktion zur Qualitätsmessung einer Aufspaltung der Knoten im Baum gewählt werden.
* Der Wert für den Parameter _max\_depth_ bestimmt die maximale Tiefe des Entscheidungsbaums.
* Mit dem Parameter _max\_leaf\_nodes_ wird bestimmt, wie viele Blätter der resultierende Entscheidungsbaum enthalten soll. Dabei werden die besten Blätter ausgewählt, d.h. Blätter, die zur relativen Verringerung der Verunreinigung am meisten beitragen.
* Der Wert des Parameters _min\_samples\_leaf_ bestimmt die minimale Anzahl der Beobachtungen in den Blättern des Entscheidungsbaums. Bei der Aufspaltung eines inneren Knotens kann passieren, dass mehr als nur ein Merkmal zur Verringerung der Verunreinigung beitragen kann. In diesem Fall wird ein Merkmal zufällig ausgewählt, auf deren Grundlage die Aufspaltung des Knotens durchgeführt wird. Mit der Wertzuweisung des Parameters _random\_state_ kann die Merkmalswahl in solchen Fällen festgelegt werden.

Das Anpassen des Modells an die Trainingsdaten erfolgt mit der Funktion _fit()_.

In [ ]:
# Erstelle das Entscheidungsbaummodell
dt_classifier = tree.DecisionTreeClassifier(max_depth=3,
                                            random_state=45)

# Trainiere das Modell mit der Trainingsmenge
dt_classifier.fit(X_train, y_train)

Mit der Funktion _predict()_ werden die Vorhersagen für alle Beobachtungen der Testmenge getroffen.

In [ ]:
# Erstelle die Vorhersage für die Testmenge
y_pred = dt_classifier.predict(X_test)

Der Entscheidungsbaum und die erlernten Regeln können mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html">_plot_tree()_</a> grafisch dargestellt werden. In der Wurzel und den inneren Knoten sind die Regeln angegeben, die bei der Aufspaltung der Beobachtungen getroffen wurden. Außerdem ist die Anzahl der Beobachtungen insgesamt und nach Klasse aufgeteilt innerhalb der inneren Knoten aufgelistet.

In [ ]:
# Plotte den Entscheidungsbaum
tree.plot_tree(dt_classifier)

Um sehen zu können, wie gut das Entscheidungsbaummodell die Klasse für neue Beobachtungen schätzen kann, wird exemplarisch für eine beliebige Beobachtung der Testmenge die tatsächliche und die durch das Klassifikationsmodell geschätzte Klasse bzw. Rebsorte ausgegeben.

In [ ]:
# Die Variable "index" enthält den Index der
# Beobachtung aus der Testmenge
index = 0

# Gebe den tatsächlichen Wert des Zielmerkmals für
# eine Beobachtung der Testmenge aus
print("Die tatsächliche Rebsorte der "
      "ersten Beobachtung aus der Testmenge:", y_test[index])

# Gebe den geschätzten Wert des Zielmerkmals für
# eine Beobachtung der Testmenge aus
print("Die geschätzte Rebsorte der "
      "ersten Beobachtung aus der Testmenge:", y_pred[index])

Mit der Funktion <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">_accuracy\_score()_</a> des Untermoduls <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics">_Metrics_</a> von _Scikit-learn_ wird die _Accuracy_ für das Entscheidungsbaummodell anhand der tatsächlichen und prognostizierten Klassen für Beobachtungen der Testmenge berechnet.

In [ ]:
# Berechne die Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In einem 3-dimensionalen Diagramm werden die Beobachtungen der Testmenge grafisch dargestellt. Dafür werden nur die Merkmale - die Farbtiefe, das Flavonoidgehalt und das Prolingehalt - verwendet. Die Rebsorten der Beobachtungen werden durch unterschiedliche Farben markiert. Dabei werden durch die Diamanten die tatsächlichen und durch Kreuze die geschätzten Klassen der Beobachtungen abgebildet.

In [ ]:
# Visualisiere die tatsächliche Zugehörigkeit als Diamant
# und die geschätzte als Kreuz
fig3 = px.scatter_3d(x=X_test[:,9], y=X_test[:,6], z=X_test[:,12],
                     color=y_test.astype(str), opacity=0.8,
                     symbol_sequence=["diamond"])
fig4 = px.scatter_3d(x=X_test[:,9], y=X_test[:,6], z=X_test[:,12],
                     color=y_pred.astype(str),
                     symbol_sequence=["cross"])
fig5 = go.Figure(data=fig3.data + fig4.data)
fig5.show()